In [1]:
import numpy as np
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [2]:
image_gen = ImageDataGenerator(rescale=1/255)

In [3]:
def genData(data_dir, batch_size):
    gen_data = image_gen.flow_from_directory(
        data_dir,
        target_size=(48,48),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale'
    )

    return gen_data

In [4]:
batch_size = 64
num_epochs = 50
train_size = 28709
test_size = 7178

train_dir = 'data/train'
test_dir = 'data/test'

In [5]:
train_data = genData(train_dir, batch_size)


Found 28709 images belonging to 7 classes.


In [6]:
test_data = genData(test_dir, batch_size)

Found 7178 images belonging to 7 classes.


In [28]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))

In [29]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
model_info = model.fit(
    train_data,
    epochs=num_epochs,
    steps_per_epoch= train_size // batch_size,
    validation_data=test_data,
    validation_steps = test_size // batch_size
)

Epoch 1/50
448/448 [==============================] - 1672s 4s/step - loss: 1.7025 - accuracy: 0.3106 - val_loss: 1.4867 - val_accuracy: 0.4268
Epoch 2/50
448/448 [==============================] - 534s 1s/step - loss: 1.3969 - accuracy: 0.4633 - val_loss: 1.2764 - val_accuracy: 0.5127
Epoch 3/50
448/448 [==============================] - 536s 1s/step - loss: 1.2662 - accuracy: 0.5168 - val_loss: 1.1859 - val_accuracy: 0.5455
Epoch 4/50
448/448 [==============================] - 549s 1s/step - loss: 1.1969 - accuracy: 0.5479 - val_loss: 1.1435 - val_accuracy: 0.5635
Epoch 5/50
448/448 [==============================] - 548s 1s/step - loss: 1.1367 - accuracy: 0.5695 - val_loss: 1.1260 - val_accuracy: 0.5751
Epoch 6/50
448/448 [==============================] - 547s 1s/step - loss: 1.0918 - accuracy: 0.5866 - val_loss: 1.1053 - val_accuracy: 0.5773
Epoch 7/50
448/448 [==============================] - 557s 1s/step - loss: 1.0468 - accuracy: 0.6037 - val_loss: 1.0751 - val_accuracy: 0.600

KeyboardInterrupt: 

In [30]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255,0,0), 2)
        roi = gray[y:y+h, x:x+w]
        crop = np.expand_dims(np.expand_dims(cv2.resize(roi, (48, 48)), -1), 0)
        # prediction = model.predict(crop)
        # emotion = int(np.argmax(prediction))
        cv2.putText(frame, "Hello", (x,y-60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 1,cv2.LINE_AA)

    cv2.imshow('Me', cv2.resize(frame, (640, 480)))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()